In [1]:
# for teleoperation test


#!/usr/bin/python
# -*- coding: utf-8 -*



"""
Control mechanical hand in virtual simulation environment(UE4/Unity/MuJoCo) or in real world scene(PR2/UR5) 
based the application protocol MQTT of IoT, this script works as a client node which connect to the broker 
agent for data processing and hand controller.
"""



import paho.mqtt.client as mqtt
import json
import time
import sys
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/drive_hand/")

from numpy.testing import assert_array_equal,assert_almost_equal
from mujoco_py import MjSim, MjViewer, load_model_from_xml, load_model_from_path, MjSimState, ignore_mujoco_warnings
import mujoco_py
import numpy as np
import cv2
from skimage.io import imsave, imshow
import matplotlib.pyplot as plt
from funtest.test_pathlib import first_try



xml_path = "/home/jade/DRL/codes/MuJoCo/Resources/xml_model/MPL/robot_hand.xml"
model = load_model_from_path(xml_path)
sim = MjSim(model)
viewer = MjViewer(sim)

wrist_j0 = sim.model.get_joint_qpos_addr("wrist_PRO")
wrist_j1 = sim.model.get_joint_qpos_addr("wrist_UDEV")
wrist_j2 = sim.model.get_joint_qpos_addr("wrist_FLEX")

thumb_j0 = sim.model.get_joint_qpos_addr("thumb_ABD")
thumb_j1 = sim.model.get_joint_qpos_addr("thumb_MCP")
thumb_j2 = sim.model.get_joint_qpos_addr("thumb_PIP")
thumb_j3 = sim.model.get_joint_qpos_addr("thumb_DIP")

index_j0 = sim.model.get_joint_qpos_addr("index_ABD")
index_j1 = sim.model.get_joint_qpos_addr("index_MCP")
index_j2 = sim.model.get_joint_qpos_addr("index_PIP")
index_j3 = sim.model.get_joint_qpos_addr("index_DIP")

middle_j0 = sim.model.get_joint_qpos_addr("middle_ABD")
middle_j1 = sim.model.get_joint_qpos_addr("middle_MCP")
middle_j2 = sim.model.get_joint_qpos_addr("middle_PIP")
middle_j3 = sim.model.get_joint_qpos_addr("middle_DIP")

ring_j0 = sim.model.get_joint_qpos_addr("ring_ABD")
ring_j1 = sim.model.get_joint_qpos_addr("ring_MCP")
ring_j2 = sim.model.get_joint_qpos_addr("ring_PIP")
ring_j3 = sim.model.get_joint_qpos_addr("ring_DIP")

pinky_j0 = sim.model.get_joint_qpos_addr("pinky_ABD")
pinky_j1 = sim.model.get_joint_qpos_addr("pinky_MCP")
pinky_j2 = sim.model.get_joint_qpos_addr("pinky_PIP")
pinky_j3 = sim.model.get_joint_qpos_addr("pinky_DIP")



'''
construct the MQTT client for communicating with MQTT broker and realizing data transmission
'''
class MQTTClient:
    def __init__(self, client_id="MQTT_Client", broker_host="192.168.6.131", broker_port=1883, broker_user=None, broker_pass=None, client_keepalive=60):
        # define mqtt parameters
        self.client_id = client_id
        self.broker_host = broker_host
        self.broker_port = broker_port
        self.broker_user = broker_user
        self.broker_pass = broker_pass
        self.client_keepalive = client_keepalive

        # redefine the topic parameters while dealing with topic
        self.topic_sub = "msg_from_server"
        self.qos_sub = 0
        # self.msg_sub = json.dumps({"thumb": "[0.0, 0.0, 0.0, 0.0]", "index": "[0.0, 0.0, 0.0, 0.0]", "middle": "[0.0, 0.0, 0.0, 0.0]", "ring": "[0.0, 0.0, 0.0, 0.0]", "pinky": "[0.0, 0.0, 0.0, 0.0]"})
        self.msg_sub = None
        self.topic_pub = "msg_to_server"
        self.qos_pub = 0
        self.msg_pub = None

        # overwrite mqtt functions for specified usage
        self.client = mqtt.Client(self.client_id)
        self.client.on_connect = self.on_connect
        self.client.on_disconnect = self.on_disconnect
        self.client.on_subscribe = self.on_subscribe
        self.client.on_unsubscribe = self.on_unsubscribe
        self.client.on_publish = self.on_publish
        self.client.on_message = self.on_message
        self.client.on_log = self.on_log

        # change connection method and loop method
        self.connect_async = self.client.connect_async #asynchronous connect the broker in a non-blocking manner, and the the connection will not complete until loop_start() is called
        self.connect_srv = self.client.connect_srv #connect a broker using an SRV DNS lookup to obtain the broker address
        self.loop = self.client.loop
        self.loop_start = self.client.loop_start
        self.loop_stop = self.client.loop_stop
        self.loop_forever = self.client.loop_forever #blocking network and will not return until the client calls disconnect()


    def connect(self):
        self.client.connect(self.broker_host, self.broker_port, self.client_keepalive)
        print("Connecting to server[%s]......"%self.broker_host)


    def on_connect(self, client, userdata, flags, rc):
        print("Connected to sever[%s] with result code "%self.broker_host + str(rc))


    def disconnect(self):
        self.client.disconnect()
        print("Disconnecting from server[%s]......"%self.broker_host)
    

    def on_disconnect(self, client, userdata, rc):
        print("Disconnected from server[%s] with result code "%self.broker_host + str(rc))
    

    # @staticmethod
    def subscribe(self):
        self.client.subscribe(self.topic_sub, self.qos_sub)
        print("Subscribing topic[%s] from server[%s]......"%(self.topic_sub, self.broker_host))


    def on_subscribe(self, client, userdata, mid, granted_qos):
        print("Subscribed topic[%s] form server[%s]: "%(self.topic_sub, self.broker_host) + str(mid) + " " + str(granted_qos))


    def unsubscribe(self):
        self.client.unsubscribe(self.topic_sub)
        print("Unsubscribing topic[%s] from server[%s]......"%(self.topic_sub, self.broker_host))
    

    def on_unsubscribe(self, client, userdata, mid):
        print("Unsubscribed topic[%s] from server[%s]: "%(self.topic_sub, self.broker_host) + str(mid) + " " + str(self.qos_sub))    
    # @staticmethod
    def publish(self):
        self.client.publish(self.topic_pub, payload=self.msg_pub, qos=self.qos_pub, retain=False)
        print("Publishing topic[%s] to server[%s]......"%(self.topic_pub, self.broker_host))


    def on_publish(self, client, userdata, mid):
        print("Published topic[%s] to server[%s]: "%(self.topic_pub, self.broker_host) + str(mid))


    def on_message(self, client, userdata, msg):
        try:
            self.msg_sub=json.loads(msg.payload)
            print("Received msg[%s] from server[%s]"%(self.msg_sub, self.broker_host))
        except json.JSONDecodeError as e:
            print("Parsing Msg Error: %s"%e.msg)
    

    def on_log(self, client, userdata, level, string):
        print(string)


        
def print_hand_joint_pose(sim):
    print("hand pose:")
    print("wrist=", sim.data.get_joint_qpos("wrist_PRO"), sim.data.get_joint_qpos("wrist_UDEV"), sim.data.get_joint_qpos("wrist_FLEX"))
    print("thumb=", sim.data.get_joint_qpos("thumb_ABD"), sim.data.get_joint_qpos("thumb_MCP"), sim.data.get_joint_qpos("thumb_PIP"), sim.data.get_joint_qpos("thumb_DIP"))
    print("index=", sim.data.get_joint_qpos("index_ABD"), sim.data.get_joint_qpos("index_MCP"), sim.data.get_joint_qpos("index_PIP"), sim.data.get_joint_qpos("index_DIP"))
    print("middle=", sim.data.get_joint_qpos("middle_ABD"), sim.data.get_joint_qpos("middle_MCP"), sim.data.get_joint_qpos("middle_PIP"), sim.data.get_joint_qpos("middle_DIP"))
    print("ring=", sim.data.get_joint_qpos("ring_ABD"), sim.data.get_joint_qpos("ring_MCP"), sim.data.get_joint_qpos("ring_PIP"), sim.data.get_joint_qpos("ring_DIP"))
    print("pinky=", sim.data.get_joint_qpos("pinky_ABD"), sim.data.get_joint_qpos("pinky_MCP"), sim.data.get_joint_qpos("pinky_PIP"), sim.data.get_joint_qpos("pinky_DIP"))

        
    
"""
Construct the instance for the MQTT Client and communicate with the broker server for data transmission
"""
# instance the mqtt client and connect to the broker server
mqtt_client = MQTTClient("Jade-MuJoCo")
mqtt_client.connect()

# redefine mqtt client topic_pub/topic_sub parameters
mqtt_client.topic_pub = "ue4_states"
mqtt_client.qos_pub = 0
mqtt_client.topic_sub = "hand_pose"
mqtt_client.qos_sub = 0


# hand_pose_zero = {"thumb": [1.0, 0.0, 0.0, 0.0], "index": [1.0, 0.0, 0.0, 0.0], "middle": [0.0, 0.0, 0.0, 0.0], "ring": [0.0, 0.0, 0.0, 0.0], "pinky": [0.0, 0.0, 0.0, 0.0]}

try:
    mqtt_client.subscribe()   

    while True:
        mqtt_client.loop_start()
        
        hand_pose = mqtt_client.msg_sub
        # print(hand_pose)
        
        if hand_pose==None:
            continue
        else:
            print("haha")

            sim_state = sim.get_state()

            sim_state.qpos[wrist_j0] = 0.0
            sim_state.qpos[wrist_j1] = 0.0
            sim_state.qpos[wrist_j2] = 0.0

            sim_state.qpos[thumb_j0] = hand_pose['thumb'][0]
            sim_state.qpos[thumb_j1] = hand_pose['thumb'][1]
            sim_state.qpos[thumb_j2] = hand_pose['thumb'][2]
            sim_state.qpos[thumb_j3] = hand_pose['thumb'][3]

            sim_state.qpos[index_j0] = hand_pose['index'][0]
            sim_state.qpos[index_j1] = hand_pose['index'][1]
            sim_state.qpos[index_j2] = hand_pose['index'][2]
            sim_state.qpos[index_j3] = hand_pose['index'][3]

            sim_state.qpos[middle_j0] = hand_pose['middle'][0]
            sim_state.qpos[middle_j1] = hand_pose['middle'][1]
            sim_state.qpos[middle_j2] = hand_pose['middle'][2]
            sim_state.qpos[middle_j3] = hand_pose['middle'][3]

            sim_state.qpos[ring_j0] = hand_pose['ring'][0]
            sim_state.qpos[ring_j1] = hand_pose['ring'][1]
            sim_state.qpos[ring_j2] = hand_pose['ring'][2]
            sim_state.qpos[ring_j3] = hand_pose['ring'][3]

            sim_state.qpos[pinky_j0] = hand_pose['pinky'][0]
            sim_state.qpos[pinky_j1] = hand_pose['pinky'][1]
            sim_state.qpos[pinky_j2] = hand_pose['pinky'][2]
            sim_state.qpos[pinky_j3] = hand_pose['pinky'][3]

            sim.set_state(sim_state)

            sim.forward()
            sim.step()

            viewer.render()
#             time.sleep(0.1)

finally:
    mqtt_client.disconnect()
    print("what the fuck")



Creating window glfw
Sending CONNECT (u0, p0, wr0, wq0, wf0, c1, k60) client_id=b'Jade-MuJoCo'
Connecting to server[192.168.6.131]......
Sending SUBSCRIBE (d0, m1) [(b'hand_pose', 0)]
Subscribing topic[hand_pose] from server[192.168.6.131]......
Received CONNACK (0, 0)
Connected to sever[192.168.6.131] with result code 0
Received SUBACK
Subscribed topic[hand_pose] form server[192.168.6.131]: 1 (0,)
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (217 bytes)
Received msg[{'thumb': [0.0, 0.4326314330101013, 1.03, 1.28], 'index': [0.06250334531068802, 1.57, 1.72, 1.38], 'middle': [0.2030162811279297, 1.57, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]haha

Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (216 bytes)
Received msg[{'thumb': [0.0, 0.644871711730957, 1.03, 1.28], 'index': [0.09075158834457397, 1.57, 1.72, 1.38], 'middle': [0.2638528048992157, 1.57, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.

haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
hahaReceived PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (218 bytes)
haha
haha

Received msg[{'thumb': [0.0, 0.6252695322036743, 1.03, 1.28], 'index': [0.06786786764860153, 1.57, 1.72, 1.38], 'middle': [0.26116663217544556, 1.57, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]haha

haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (218 bytes)haha

Received msg[{'thumb': [0.0, 0.5536168217658997, 1.03, 1.28], 'index':

haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
hahaReceived PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (216 bytes)
haha

Received msg[{'thumb': [0.0, 0.556806206703186, 1.03, 1.28], 'index': [0.07231248170137405, 1.57, 1.72, 1.38], 'middle': [0.2157711237668991, 1.57, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (230 bytes)
Received msg[{'thumb': [0.0, 0.20403780043125153, 1.03, 1.28], 'index': [0.0, 1.57, 1.72, 1.38], 'middle': [0.0, 1.57, 1.2203599214553833, 0.0], 'ring': [0.1142059788107872, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.320376992225647]

haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (218 bytes)
Received msg[{'thumb': [0.0, 0.5903792381286621, 1.03, 1.28], 'index': [0.12182687222957611, 1.57, 1.72, 1.38], 'middle': [0.33565548062324524, 1.57, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (234 bytes)
Received msg[{'thumb': [0.0, 0.2613260746002197, 1.03, 1.28], 'in

Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (204 bytes)
Received msg[{'thumb': [0.0, 1.03, 1.03, 1.28], 'index': [0.17470072209835052, 1.57, 1.72, 1.38], 'middle': [0.345, 0.9553791880607605, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (206 bytes)
Received msg[{'thumb': [0.0, 0.8350376486778259, 1.03, 1.28], 'index': [0.21789191663265228, 1.57, 1.72, 1.38], 'middle': [0.345, -0.785, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]
haha
haha
haha
h

haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (217 bytes)
Received msg[{'thumb': [0.0, 0.5610523223876953, 1.03, 1.28], 'index': [0.0474737323820591, 1.57, 1.72, 1.38], 'middle': [0.16413795948028564, 1.57, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
h

Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (205 bytes)
Received msg[{'thumb': [0.0, 0.996213972568512, 1.03, 1.28], 'index': [0.11554769426584244, 1.57, 1.72, 1.38], 'middle': [0.345, -0.785, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
hahaReceived PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (216 bytes)
Received msg[{'thumb': [0.0, 0.7188800573348999, 1.03, 1.28], 'index': [0.1059834212064743, 1.57, 1.72, 1.38], 'middle': [0.2996809184551239, 1.57, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]

haha
haha
haha
haha
haha


haha
haha
haha
haha
haha
haha
haha
haha
haha
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (203 bytes)haha

haha
Received msg[{'thumb': [0.0, 0.4071425497531891, 1.03, 1.28], 'index': [0.0, 1.57, 1.72, 1.38], 'middle': [0.010575498454272747, 1.57, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]haha

haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (230 bytes)
Received msg[{'thumb': [0.0, 0.5891370177268982, 1.03, 1.28], 'index': [0.015151642262935638, 1.57, 1.72, 0.0], 'middle': [0.0, 1.57, 0.8047612309455872, 0.0], 'ring': [0.0, 1.57, 1.397830605506897, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.

haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (206 bytes)
Received msg[{'thumb': [0.0, 0.9995421767234802, 1.03, 1.28], 'index': [0.10552556067705154, 1.57, 1.72, 1.38], 'middle': [0.345, -0.785, 0.0, 0.0], 'ring': [0.0, 1.57, 1.72, 1.38], 'pinky': [0.345, -0.785, 1.72, 1.38]}] from server[192.168.6.131]
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
Received PUBLISH (d0, q0, r0, m0), 'hand_pose', ...  (231 bytes)
Received msg[{'thumb': [0.0, 0.5956308841705322, 1.03, 1.28], 'index': [0.07253705

NameError: name 'exit' is not defined

In [ ]:
import numpy as np
a = np.array([1,2,3])
b=float(a)
print(b)
print(type(a))
print(a)
print(a.shape)

In [ ]:
str = '1,2,3'
arr = str.split(',')
print(arr)

arr = ['a','b']
str = ','.join(arr)
print(str)

In [ ]:
# test for dataloader

import os
import numpy as np
import cv2


path = "/home/jade/DRL/codes/MuJoCo/Hand_Control/TeachNet-Zenmme/dataset/"
# pic = cv2.imread(os.path.join(path,'human/human_crop/','P0_0_crop'),cv2.IMREAD_ANYDEPTH)
# cv2.imshow('haha',pic)
file = np.loadtxt(os.path.join(path, "mpl/mpl_mujoco/depth_mpl0", "hand_calculated_joints.txt"), dtype=str)
# print(file)
print(len(file))

content = file[0]
# print(type(content))
# print(content)
# str1 = str(content)
# print(type(str1))
# print(str1)

str2 = ''.join(content)
# print(type(str2))
# print(str2)

# a = str2.split('-')
# print(type(a))
# print(a)
str3 = str2.split(':',1)
# print(type(str3[1]))
# print(str3[1])
angle = eval(str3[1])
# print(angle)
# print(type(angle))
temp = float(np.array([angle["thumb"],angle["index"],angle["middle"],angle["ring"],angle["pinky"]]))
print(type(temp))
print(temp)
print(temp.shape)
fin = temp.reshape(20,)
print(fin)
print(type(fin))
print(fin.shape)


# str1 = ' '.join(content)
# print(str1)
# fname = content[0]
# print(file[0][0])
# cv2.waitKey(0)


In [ ]:
# test for the joint position calculator

# import, choose and show data sample
%matplotlib widget
import sys
import numpy as np
# sys.path.append("/data/msra_importer/")
from data_importer.importer import Anyone

# idx = np.random.randint(3000)
# ay = Anyone(idx)
ay = Anyone(15)

idx=20
ay.draw_point(idx)
finger_data = ay.data.gtorig
print(finger_data)


# dynamic show the 3d hand keypoints pos for different viewangle
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time

# create a figure for image operation
fig = plt.figure(dpi=150)
ax = fig.add_subplot(111, projection='3d')

# prepare the 3d finger keypoints data(sequence:thumb/index/middle/ring/pinky)
x1 = np.append(finger_data[0][0],[k[0] for k in finger_data[17:21]])
y1 = np.append(finger_data[0][1],[k[1] for k in finger_data[17:21]])
z1 = np.append(finger_data[0][2],[k[2] for k in finger_data[17:21]])

x2 = np.append(finger_data[0][0],[k[0] for k in finger_data[1:5]])
y2 = np.append(finger_data[0][1],[k[1] for k in finger_data[1:5]])
z2 = np.append(finger_data[0][2],[k[2] for k in finger_data[1:5]])

x3 = np.append(finger_data[0][0],[k[0] for k in finger_data[5:9]])
y3 = np.append(finger_data[0][1],[k[1] for k in finger_data[5:9]])
z3 = np.append(finger_data[0][2],[k[2] for k in finger_data[5:9]])

x4 = np.append(finger_data[0][0],[k[0] for k in finger_data[9:13]])
y4 = np.append(finger_data[0][1],[k[1] for k in finger_data[9:13]])
z4 = np.append(finger_data[0][2],[k[2] for k in finger_data[9:13]])

x5 = np.append(finger_data[0][0],[k[0] for k in finger_data[13:17]])
y5 = np.append(finger_data[0][1],[k[1] for k in finger_data[13:17]])
z5 = np.append(finger_data[0][2],[k[2] for k in finger_data[13:17]])

# plot and rotate
wframe1 = None
tstart = time.time()

# If a line collection is already here and then remove it before drawing.
if wframe1:
    ax.collections.remove(wframe1)
    ax.collections.remove(wframe2)
    ax.collections.remove(wframe3)
    ax.collections.remove(wframe4)
    ax.collections.remove(wframe5)

# Plot the new wireframe and pause briefly before continuing.
plt.title('Hand Keypoints')
wframe1 = ax.plot(x1,y1,z1,c='r',marker='.',linewidth=1,alpha=1,label='thumb')
wframe2 = ax.plot(x2,y2,z2,c='g',marker='.',linewidth=1,alpha=1,label='index')
wframe3 = ax.plot(x3,y3,z3,c='b',marker='.',linewidth=1,alpha=1,label='middle')
wframe4 = ax.plot(x4,y4,z4,c='y',marker='.',linewidth=1,alpha=1,label='ring')
wframe5 = ax.plot(x5,y5,z5,c='k',marker='.',linewidth=1,alpha=1,label='pinky')

# configure the plot setting
# ax.axis('scaled')
ax.axis('image')
ax.set_xlabel('x pos')
ax.set_ylabel('y pos')
ax.set_zlabel('z pos')

# plt.pause(.001)

print('Average FPS: %f' % (100 / (time.time() - tstart)))


# test for the mapping data from the human hand to the model hand MPL/Adroit
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/drive_hand")
from hand_mapping import hand_keypoints_mapping

print("figner_data:\n%s"%finger_data)
finger_data_mapping = hand_keypoints_mapping.calc_model_mpl_keypoints(finger_data)
print("figner_data_mapping:\n%s"%finger_data_mapping)

# from joint_position_calculator import get_pose_new5
# hand_pose = get_pose_new4.get_hand_joint_pose(finger_data)
# print(hand_pose)
from joint_position_calculator import get_pose_new4
hand_pose = get_pose_new4.get_hand_joint_pose(finger_data)
print("hand_pose:\n%s"%hand_pose)


# test example for show the result
from numpy.testing import assert_array_equal,assert_almost_equal
from mujoco_py import MjSim, MjViewer, load_model_from_xml, load_model_from_path, MjSimState, ignore_mujoco_warnings
import mujoco_py
import numpy as np
import cv2
from skimage.io import imsave, imshow
import matplotlib.pyplot as plt
import json
from funtest.test_pathlib import first_try
import time

xml_path = "/home/jade/DRL/codes/MuJoCo/xml_model/MPL/robot_hand.xml"
model = load_model_from_path(xml_path)
sim = MjSim(model)
viewer = MjViewer(sim)
sim.forward()

def print_hand_joint_pose(sim):
    print("hand pose:")
    print("wrist=", sim.data.get_joint_qpos("wrist_PRO"), sim.data.get_joint_qpos("wrist_UDEV"), sim.data.get_joint_qpos("wrist_FLEX"))
    print("thumb=", sim.data.get_joint_qpos("thumb_ABD"), sim.data.get_joint_qpos("thumb_MCP"), sim.data.get_joint_qpos("thumb_PIP"), sim.data.get_joint_qpos("thumb_DIP"))
    print("index=", sim.data.get_joint_qpos("index_ABD"), sim.data.get_joint_qpos("index_MCP"), sim.data.get_joint_qpos("index_PIP"), sim.data.get_joint_qpos("index_DIP"))
    print("middle=", sim.data.get_joint_qpos("middle_ABD"), sim.data.get_joint_qpos("middle_MCP"), sim.data.get_joint_qpos("middle_PIP"), sim.data.get_joint_qpos("middle_DIP"))
    print("ring=", sim.data.get_joint_qpos("ring_ABD"), sim.data.get_joint_qpos("ring_MCP"), sim.data.get_joint_qpos("ring_PIP"), sim.data.get_joint_qpos("ring_DIP"))
    print("pinky=", sim.data.get_joint_qpos("pinky_ABD"), sim.data.get_joint_qpos("pinky_MCP"), sim.data.get_joint_qpos("pinky_PIP"), sim.data.get_joint_qpos("pinky_DIP"))

    
print_hand_joint_pose(sim)

wrist_j0 = sim.model.get_joint_qpos_addr("wrist_PRO")
wrist_j1 = sim.model.get_joint_qpos_addr("wrist_UDEV")
wrist_j2 = sim.model.get_joint_qpos_addr("wrist_FLEX")

thumb_j0 = sim.model.get_joint_qpos_addr("thumb_ABD")
thumb_j1 = sim.model.get_joint_qpos_addr("thumb_MCP")
thumb_j2 = sim.model.get_joint_qpos_addr("thumb_PIP")
thumb_j3 = sim.model.get_joint_qpos_addr("thumb_DIP")

index_j0 = sim.model.get_joint_qpos_addr("index_ABD")
index_j1 = sim.model.get_joint_qpos_addr("index_MCP")
index_j2 = sim.model.get_joint_qpos_addr("index_PIP")
index_j3 = sim.model.get_joint_qpos_addr("index_DIP")

middle_j0 = sim.model.get_joint_qpos_addr("middle_ABD")
middle_j1 = sim.model.get_joint_qpos_addr("middle_MCP")
middle_j2 = sim.model.get_joint_qpos_addr("middle_PIP")
middle_j3 = sim.model.get_joint_qpos_addr("middle_DIP")

ring_j0 = sim.model.get_joint_qpos_addr("ring_ABD")
ring_j1 = sim.model.get_joint_qpos_addr("ring_MCP")
ring_j2 = sim.model.get_joint_qpos_addr("ring_PIP")
ring_j3 = sim.model.get_joint_qpos_addr("ring_DIP")

pinky_j0 = sim.model.get_joint_qpos_addr("pinky_ABD")
pinky_j1 = sim.model.get_joint_qpos_addr("pinky_MCP")
pinky_j2 = sim.model.get_joint_qpos_addr("pinky_PIP")
pinky_j3 = sim.model.get_joint_qpos_addr("pinky_DIP")

while True:
    sim_state = sim.get_state()

    sim_state.qpos[wrist_j0] = np.pi
    sim_state.qpos[wrist_j1] = 0.0
    sim_state.qpos[wrist_j2] = 0.0

    sim_state.qpos[thumb_j0] = hand_pose['thumb'][0]
    sim_state.qpos[thumb_j1] = hand_pose['thumb'][1]
    sim_state.qpos[thumb_j2] = hand_pose['thumb'][2]
    sim_state.qpos[thumb_j3] = hand_pose['thumb'][3]
    # sim_state.qpos[thumb_j0] = 0.0
    # sim_state.qpos[thumb_j1] = 0.0
    # sim_state.qpos[thumb_j2] = 0.0
    # sim_state.qpos[thumb_j3] = 0.0

    sim_state.qpos[index_j0] = hand_pose['index'][0]
    sim_state.qpos[index_j1] = hand_pose['index'][1]
    sim_state.qpos[index_j2] = hand_pose['index'][2]
    sim_state.qpos[index_j3] = hand_pose['index'][3]
    # sim_state.qpos[index_j0] = 0.0
    # sim_state.qpos[index_j1] = 0.0
    # sim_state.qpos[index_j2] = 0.0
    # sim_state.qpos[index_j3] = 0.0

    sim_state.qpos[middle_j0] = hand_pose['middle'][0]
    sim_state.qpos[middle_j1] = hand_pose['middle'][1]
    sim_state.qpos[middle_j2] = hand_pose['middle'][2]
    sim_state.qpos[middle_j3] = hand_pose['middle'][3]
    # sim_state.qpos[middle_j0] = 0.0
    # sim_state.qpos[middle_j1] = 0.0
    # sim_state.qpos[middle_j2] = 0.0
    # sim_state.qpos[middle_j3] = 0.0

    sim_state.qpos[ring_j0] = hand_pose['ring'][0]
    sim_state.qpos[ring_j1] = hand_pose['ring'][1]
    sim_state.qpos[ring_j2] = hand_pose['ring'][2]
    sim_state.qpos[ring_j3] = hand_pose['ring'][3]
    # sim_state.qpos[ring_j0] = 0.0
    # sim_state.qpos[ring_j1] = 0.0
    # sim_state.qpos[ring_j2] = 0.0
    # sim_state.qpos[ring_j3] = 0.0

    sim_state.qpos[pinky_j0] = hand_pose['pinky'][0]
    sim_state.qpos[pinky_j1] = hand_pose['pinky'][1]
    sim_state.qpos[pinky_j2] = hand_pose['pinky'][2]
    sim_state.qpos[pinky_j3] = hand_pose['pinky'][3]
    # sim_state.qpos[pinky_j0] = 0.0
    # sim_state.qpos[pinky_j1] = 0.0
    # sim_state.qpos[pinky_j2] = 0.0
    # sim_state.qpos[pinky_j3] = 0.0

    sim.set_state(sim_state)

    # sim.forward()
    sim.step()
    # assert sim.data.ncon == 0, 'Collision Detected!'
    # print("move hand to", hand_pose)
    # print_hand_joint_pose(sim)
    viewer.render()
    # assert sim.data.ncon == 0, 'Collision Detected!'
    time.sleep(0.001)



In [ ]:
# Results compare for get_pose_new4 and get_pose_new6

1234(index joint0有问题)
new6:
    {'thumb': [0.17669384679053976, 0.15694691069904643, 0.4420535485414106, 0.0, 0.0, -0.15995685390707087], 'index': [0.06054524465557379, 0.06657799297047161, 0.09251335906223598, 0.0, 0.08391872369483583], 'middle': [-0.007568509527252757, -0.006892941712684638, 0.8796202059040948, 1.9792440995428109, 0.7656219770404471], 'ring': [-0.08600329659277023, -0.10408227524301485, 1.1085364841182042, 1.9208821186921845, 0.6502685304918465], 'pinky': [0.024166169052284213, 0.4706339235699013, 1.1394005994570742, 1.434970161406652, 0.7108003960946077]}

new4:
    {'thumb': [1.5855900887747543, -0.06311890571270543, -0.03616233351822275, -0.19705418916478323], 'index': [0.060545244655574676, 0.09251335906223598, 0.11852916112012779, 0.085500480393749], 'middle': [-0.007568509527252757, 0.8796202059040951, 1.9792440995428109, 0.7658904135013614], 'ring': [0.1037459213455645, 1.1085364841182042, 1.9208821186921845, 0.6508244465661344], 'pinky': [0.47518921730848024, 1.1394005994570742, 1.434970161406652, 0.7108144987103008]}

# Appendix

In [ ]:
# test for mqtt communication

#!/usr/bin/python
# -*- coding: utf-8 -*

"""
MQTT Client Test for communication 
"""
import paho.mqtt.client as mqtt
import json
import time


'''
construct the MQTT client for communicating with MQTT broker and realizing data transmission
'''
class MQTTClient:
    def __init__(self, client_id="MQTT_Client", broker_host="192.168.6.131", broker_port=1883, broker_user=None, broker_pass=None, client_keepalive=60):
        # define mqtt parameters
        self.client_id = client_id
        self.broker_host = broker_host
        self.broker_port = broker_port
        self.broker_user = broker_user
        self.broker_pass = broker_pass
        self.client_keepalive = client_keepalive

        # redefine the topic parameters while dealing with topic
        self.topic_sub = "msg_from_server"
        self.qos_sub = 0
        self.msg_sub = None
        self.topic_pub = "msg_to_server"
        self.qos_pub = 0
        self.msg_pub = None

        # overwrite mqtt functions for specified usage
        self.client = mqtt.Client(self.client_id)
        self.client.on_connect = self.on_connect
        self.client.on_disconnect = self.on_disconnect
        self.client.on_subscribe = self.on_subscribe
        self.client.on_unsubscribe = self.on_unsubscribe
        self.client.on_publish = self.on_publish
        self.client.on_message = self.on_message
        self.client.on_log = self.on_log

        # change connection method and loop method
        self.connect_async = self.client.connect_async #asynchronous connect the broker in a non-blocking manner, and the the connection will not complete until loop_start() is called
        self.connect_srv = self.client.connect_srv #connect a broker using an SRV DNS lookup to obtain the broker address
        self.loop = self.client.loop
        self.loop_start = self.client.loop_start
        self.loop_stop = self.client.loop_stop
        self.loop_forever = self.client.loop_forever #blocking network and will not return until the client calls disconnect()


    def connect(self):
        self.client.connect(self.broker_host, self.broker_port, self.client_keepalive)
        print("Connecting to server[%s]......"%self.broker_host)


    def on_connect(self, client, userdata, flags, rc):
        print("Connected to sever[%s] with result code "%self.broker_host + str(rc))


    def disconnect(self):
        self.client.disconnect()
        print("Disconnecting from server[%s]......"%self.broker_host)
    

    def on_disconnect(self, client, userdata, rc):
        print("Disconnected from server[%s] with result code "%self.broker_host + str(rc))
    

    # @staticmethod
    def subscribe(self):
        self.client.subscribe(self.topic_sub, self.qos_sub)
        print("Subscribing topic[%s] from server[%s]......"%(self.topic_sub, self.broker_host))


    def on_subscribe(self, client, userdata, mid, granted_qos):
        print("Subscribed topic[%s] form server[%s]: "%(self.topic_sub, self.broker_host) + str(mid) + " " + str(granted_qos))


    def unsubscribe(self):
        self.client.unsubscribe(self.topic_sub)
        print("Unsubscribing topic[%s] from server[%s]......"%(self.topic_sub, self.broker_host))
    

    def on_unsubscribe(self, client, userdata, mid):
        print("Unsubscribed topic[%s] from server[%s]: "%(self.topic_sub, self.broker_host) + str(mid) + " " + str(self.qos_sub))
    
    # @staticmethod
    def publish(self):
        self.client.publish(self.topic_pub, payload=self.msg_pub, qos=self.qos_pub, retain=False)
        print("Publishing topic[%s] to server[%s]......"%(self.topic_pub, self.broker_host))


    def on_publish(self, client, userdata, mid):
        print("Published topic[%s] to server[%s]: "%(self.topic_pub, self.broker_host) + str(mid))


    def on_message(self, client, userdata, msg):
        try:
            self.msg_sub=json.loads(msg.payload)
            print("Received msg[%s] from server[%s]"%(self.msg_sub, self.broker_host))
        except json.JSONDecodeError as e:
            print("Parsing Msg Error: %s"%e.msg)
        
        # return self.msg_sub
    

    def on_log(self, client, userdata, level, string):
        print(string)
        
    
"""
Construct the instance for the MQTT Client and communicate with the broker server 
"""
# instance the mqtt client and connect to the broker server
mqtt_client = MQTTClient("Jade-PC")
mqtt_client.connect()

# redefine mqtt client topic_pub/topic_sub parameters
mqtt_client.topic_pub = "hand_pose"
mqtt_client.qos_pub = 0
mqtt_client.topic_sub = "ue4_states"
mqtt_client.qos_sub = 0

# mqtt_client.subscribe()
# mqtt_client.loop()
hand_pose = {'thumb':[1,2,3,4],'index':[2,3,4,5]}
mqtt_client.msg_pub = json.dumps(hand_pose)
mqtt_client.publish()
mqtt_client.loop_start()

# while True:
#     # hand_pose = {'thumb':[1,2,3,4],'index':[2,3,4,5]}
#     # mqtt_client.msg_pub = json.dumps(hand_pose)
#     # mqtt_client.publish()
#     mqtt_client.subscribe()
#     msg_sub = mqtt_client.on_message()
#     # print(json.loads(msg_sub))
#     time.sleep(0.3)
# 



In [ ]:
# test for showing the thumb_base_pnv

%matplotlib widget
import sys
import numpy as np
# sys.path.append("/data/msra_importer/")
from data_importer.importer import Anyone

# idx = np.random.randint(3000)
# ay = Anyone(idx)
ay = Anyone(1818)

idx=20
ay.draw_point(idx)
finger_data = ay.data.gtorig
print(finger_data)


import sys
import pyquaternion as pq
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/drive_hand")
from joint_position_calculator import get_pose_new6

hand_links = get_pose_new6.get_hand_links(finger_data)
ref_pnvs = get_pose_new6.get_ref_plane_normal_vectors(finger_data)
z_axis = ref_pnvs[5] #up direction
y_axis = get_pose_new6.calc_vector_projection_onto_plane(hand_links['middle'][0], z_axis) #forward direction
x_axis = np.cross(y_axis, z_axis) #right direction
x_axis_rot = pq.Quaternion(axis=z_axis, angle=np.pi/4).rotate(x_axis)
thumb_base_pnv = pq.Quaternion(axis=x_axis_rot, angle=np.pi/4).rotate(z_axis) #calc the thumb_base_pnv
mod = np.sqrt(np.square(thumb_base_pnv[0])+np.square(thumb_base_pnv[1])+np.square(thumb_base_pnv[2]))
thumb_base_pnv_normal = thumb_base_pnv/

print(x_axis)
print(y_axis)
print(z_axis)


# dynamic show the 3d hand keypoints pos for different viewangle
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time

# create a figure for image operation
fig = plt.figure(dpi=150)
ax = fig.add_subplot(111, projection='3d')

# prepare the 3d finger keypoints data(sequence:thumb/index/middle/ring/pinky)
x1 = np.append(finger_data[0][0],[k[0] for k in finger_data[17:21]])
y1 = np.append(finger_data[0][1],[k[1] for k in finger_data[17:21]])
z1 = np.append(finger_data[0][2],[k[2] for k in finger_data[17:21]])

x2 = np.append(finger_data[0][0],[k[0] for k in finger_data[1:5]])
y2 = np.append(finger_data[0][1],[k[1] for k in finger_data[1:5]])
z2 = np.append(finger_data[0][2],[k[2] for k in finger_data[1:5]])

x3 = np.append(finger_data[0][0],[k[0] for k in finger_data[5:9]])
y3 = np.append(finger_data[0][1],[k[1] for k in finger_data[5:9]])
z3 = np.append(finger_data[0][2],[k[2] for k in finger_data[5:9]])

x4 = np.append(finger_data[0][0],[k[0] for k in finger_data[9:13]])
y4 = np.append(finger_data[0][1],[k[1] for k in finger_data[9:13]])
z4 = np.append(finger_data[0][2],[k[2] for k in finger_data[9:13]])

x5 = np.append(finger_data[0][0],[k[0] for k in finger_data[13:17]])
y5 = np.append(finger_data[0][1],[k[1] for k in finger_data[13:17]])
z5 = np.append(finger_data[0][2],[k[2] for k in finger_data[13:17]])

x6 = np.append(finger_data[0][0],thumb_base_pnv_normal[0])
y6 = np.append(finger_data[0][1],thumb_base_pnv_normal[1])
z6 = np.append(finger_data[0][2],thumb_base_pnv_normal[2])

# plot and rotate
wframe1 = None
tstart = time.time()

# If a line collection is already here and then remove it before drawing.
if wframe1:
    ax.collections.remove(wframe1)
    ax.collections.remove(wframe2)
    ax.collections.remove(wframe3)
    ax.collections.remove(wframe4)
    ax.collections.remove(wframe5)

# Plot the new wireframe and pause briefly before continuing.
plt.title('Hand Keypoints')
wframe1 = ax.plot(x1,y1,z1,c='r',marker='.',linewidth=1,alpha=1,label='thumb')
wframe2 = ax.plot(x2,y2,z2,c='g',marker='.',linewidth=1,alpha=1,label='index')
wframe3 = ax.plot(x3,y3,z3,c='b',marker='.',linewidth=1,alpha=1,label='middle')
wframe4 = ax.plot(x4,y4,z4,c='y',marker='.',linewidth=1,alpha=1,label='ring')
wframe5 = ax.plot(x5,y5,z5,c='k',marker='.',linewidth=1,alpha=1,label='pinky')
thumb_pnv = ax.plot(x6,y6,z6,c='r',marker='*',linewidth=1,alpha=1,label='thumb_base_pnv')

# configure the plot setting
# ax.axis('scaled')
ax.axis('scaled')
ax.set_xlabel('x pos')
ax.set_ylabel('y pos')
ax.set_zlabel('z pos')



In [ ]:
# test for the joint position calculator

# import, choose and show data sample
%matplotlib widget
import sys
import numpy as np
# sys.path.append("/data/msra_importer/")
from data_importer.importer import Anyone

# idx = np.random.randint(3000)
# ay = Anyone(idx)
ay = Anyone(1818)

idx=20
ay.draw_point(idx)
finger_data = ay.data.gtorig
print(finger_data)


# dynamic show the 3d hand keypoints pos for different viewangle
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time

# create a figure for image operation
fig = plt.figure(dpi=150)
ax = fig.add_subplot(111, projection='3d')

# prepare the 3d finger keypoints data(sequence:thumb/index/middle/ring/pinky)
x1 = np.append(finger_data[0][0],[k[0] for k in finger_data[17:21]])
y1 = np.append(finger_data[0][1],[k[1] for k in finger_data[17:21]])
z1 = np.append(finger_data[0][2],[k[2] for k in finger_data[17:21]])

x2 = np.append(finger_data[0][0],[k[0] for k in finger_data[1:5]])
y2 = np.append(finger_data[0][1],[k[1] for k in finger_data[1:5]])
z2 = np.append(finger_data[0][2],[k[2] for k in finger_data[1:5]])

x3 = np.append(finger_data[0][0],[k[0] for k in finger_data[5:9]])
y3 = np.append(finger_data[0][1],[k[1] for k in finger_data[5:9]])
z3 = np.append(finger_data[0][2],[k[2] for k in finger_data[5:9]])

x4 = np.append(finger_data[0][0],[k[0] for k in finger_data[9:13]])
y4 = np.append(finger_data[0][1],[k[1] for k in finger_data[9:13]])
z4 = np.append(finger_data[0][2],[k[2] for k in finger_data[9:13]])

x5 = np.append(finger_data[0][0],[k[0] for k in finger_data[13:17]])
y5 = np.append(finger_data[0][1],[k[1] for k in finger_data[13:17]])
z5 = np.append(finger_data[0][2],[k[2] for k in finger_data[13:17]])

# plot and rotate
wframe1 = None
tstart = time.time()

# If a line collection is already here and then remove it before drawing.
if wframe1:
    ax.collections.remove(wframe1)
    ax.collections.remove(wframe2)
    ax.collections.remove(wframe3)
    ax.collections.remove(wframe4)
    ax.collections.remove(wframe5)

# Plot the new wireframe and pause briefly before continuing.
plt.title('Hand Keypoints')
wframe1 = ax.plot(x1,y1,z1,c='r',marker='.',linewidth=1,alpha=1,label='thumb')
wframe2 = ax.plot(x2,y2,z2,c='g',marker='.',linewidth=1,alpha=1,label='index')
wframe3 = ax.plot(x3,y3,z3,c='b',marker='.',linewidth=1,alpha=1,label='middle')
wframe4 = ax.plot(x4,y4,z4,c='y',marker='.',linewidth=1,alpha=1,label='ring')
wframe5 = ax.plot(x5,y5,z5,c='k',marker='.',linewidth=1,alpha=1,label='pinky')

# configure the plot setting
# ax.axis('scaled')
ax.axis('image')
ax.set_xlabel('x pos')
ax.set_ylabel('y pos')
ax.set_zlabel('z pos')

# plt.pause(.001)

print('Average FPS: %f' % (100 / (time.time() - tstart)))


# test for the mapping data from the human hand to the model hand MPL/Adroit
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/drive_hand")
from hand_mapping import hand_keypoints_mapping

print("figner_data:\n%s"%finger_data)
finger_data_mapping = hand_keypoints_mapping.calc_model_mpl_keypoints(finger_data)
print("figner_data_mapping:\n%s"%finger_data_mapping)

# from joint_position_calculator import get_pose_new5
# hand_pose = get_pose_new4.get_hand_joint_pose(finger_data)
# print(hand_pose)
from joint_position_calculator import get_pose_new6
hand_pose = get_pose_new6.get_hand_joint_pose(finger_data)
print("hand_pose:\n%s"%hand_pose)


# test example for show the result
from numpy.testing import assert_array_equal,assert_almost_equal
from mujoco_py import MjSim, MjViewer, load_model_from_xml, load_model_from_path, MjSimState, ignore_mujoco_warnings
import mujoco_py
import numpy as np
import cv2
from skimage.io import imsave, imshow
import matplotlib.pyplot as plt
import json
from funtest.test_pathlib import first_try
import time

xml_path = "/home/jade/DRL/codes/MuJoCo/xml_model/MPL/robot_hand.xml"
model = load_model_from_path(xml_path)
sim = MjSim(model)
viewer = MjViewer(sim)
sim.forward()

def print_hand_joint_pose(sim):
    print("hand pose:")
    print("wrist=", sim.data.get_joint_qpos("wrist_PRO"), sim.data.get_joint_qpos("wrist_UDEV"), sim.data.get_joint_qpos("wrist_FLEX"))
    print("thumb=", sim.data.get_joint_qpos("thumb_ABD"), sim.data.get_joint_qpos("thumb_MCP"), sim.data.get_joint_qpos("thumb_PIP"), sim.data.get_joint_qpos("thumb_DIP"))
    print("index=", sim.data.get_joint_qpos("index_ABD"), sim.data.get_joint_qpos("index_MCP"), sim.data.get_joint_qpos("index_PIP"), sim.data.get_joint_qpos("index_DIP"))
    print("middle=", sim.data.get_joint_qpos("middle_ABD"), sim.data.get_joint_qpos("middle_MCP"), sim.data.get_joint_qpos("middle_PIP"), sim.data.get_joint_qpos("middle_DIP"))
    print("ring=", sim.data.get_joint_qpos("ring_ABD"), sim.data.get_joint_qpos("ring_MCP"), sim.data.get_joint_qpos("ring_PIP"), sim.data.get_joint_qpos("ring_DIP"))
    print("pinky=", sim.data.get_joint_qpos("pinky_ABD"), sim.data.get_joint_qpos("pinky_MCP"), sim.data.get_joint_qpos("pinky_PIP"), sim.data.get_joint_qpos("pinky_DIP"))

    
print_hand_joint_pose(sim)

wrist_j0 = sim.model.get_joint_qpos_addr("wrist_PRO")
wrist_j1 = sim.model.get_joint_qpos_addr("wrist_UDEV")
wrist_j2 = sim.model.get_joint_qpos_addr("wrist_FLEX")

thumb_j0 = sim.model.get_joint_qpos_addr("thumb_ABD")
thumb_j1 = sim.model.get_joint_qpos_addr("thumb_MCP")
thumb_j2 = sim.model.get_joint_qpos_addr("thumb_PIP")
thumb_j3 = sim.model.get_joint_qpos_addr("thumb_DIP")

index_j0 = sim.model.get_joint_qpos_addr("index_ABD")
index_j1 = sim.model.get_joint_qpos_addr("index_MCP")
index_j2 = sim.model.get_joint_qpos_addr("index_PIP")
index_j3 = sim.model.get_joint_qpos_addr("index_DIP")

middle_j0 = sim.model.get_joint_qpos_addr("middle_ABD")
middle_j1 = sim.model.get_joint_qpos_addr("middle_MCP")
middle_j2 = sim.model.get_joint_qpos_addr("middle_PIP")
middle_j3 = sim.model.get_joint_qpos_addr("middle_DIP")

ring_j0 = sim.model.get_joint_qpos_addr("ring_ABD")
ring_j1 = sim.model.get_joint_qpos_addr("ring_MCP")
ring_j2 = sim.model.get_joint_qpos_addr("ring_PIP")
ring_j3 = sim.model.get_joint_qpos_addr("ring_DIP")

pinky_j0 = sim.model.get_joint_qpos_addr("pinky_ABD")
pinky_j1 = sim.model.get_joint_qpos_addr("pinky_MCP")
pinky_j2 = sim.model.get_joint_qpos_addr("pinky_PIP")
pinky_j3 = sim.model.get_joint_qpos_addr("pinky_DIP")

while True:
    sim_state = sim.get_state()

    sim_state.qpos[wrist_j0] = np.pi
    sim_state.qpos[wrist_j1] = 0.0
    sim_state.qpos[wrist_j2] = 0.0

    sim_state.qpos[thumb_j0] = hand_pose['thumb'][0]
    sim_state.qpos[thumb_j1] = hand_pose['thumb'][1]
    sim_state.qpos[thumb_j2] = hand_pose['thumb'][2]
    sim_state.qpos[thumb_j3] = hand_pose['thumb'][3]
    # sim_state.qpos[thumb_j0] = 0.0
    # sim_state.qpos[thumb_j1] = 0.0
    # sim_state.qpos[thumb_j2] = 0.0
    # sim_state.qpos[thumb_j3] = 0.0

    sim_state.qpos[index_j0] = hand_pose['index'][0]
    sim_state.qpos[index_j1] = hand_pose['index'][1]
    sim_state.qpos[index_j2] = hand_pose['index'][2]
    sim_state.qpos[index_j3] = hand_pose['index'][3]
    # sim_state.qpos[index_j0] = 0.0
    # sim_state.qpos[index_j1] = 0.0
    # sim_state.qpos[index_j2] = 0.0
    # sim_state.qpos[index_j3] = 0.0

    sim_state.qpos[middle_j0] = hand_pose['middle'][0]
    sim_state.qpos[middle_j1] = hand_pose['middle'][1]
    sim_state.qpos[middle_j2] = hand_pose['middle'][2]
    sim_state.qpos[middle_j3] = hand_pose['middle'][3]
    # sim_state.qpos[middle_j0] = 0.0
    # sim_state.qpos[middle_j1] = 0.0
    # sim_state.qpos[middle_j2] = 0.0
    # sim_state.qpos[middle_j3] = 0.0

    sim_state.qpos[ring_j0] = hand_pose['ring'][0]
    sim_state.qpos[ring_j1] = hand_pose['ring'][1]
    sim_state.qpos[ring_j2] = hand_pose['ring'][2]
    sim_state.qpos[ring_j3] = hand_pose['ring'][3]
    # sim_state.qpos[ring_j0] = 0.0
    # sim_state.qpos[ring_j1] = 0.0
    # sim_state.qpos[ring_j2] = 0.0
    # sim_state.qpos[ring_j3] = 0.0

    sim_state.qpos[pinky_j0] = hand_pose['pinky'][0]
    sim_state.qpos[pinky_j1] = hand_pose['pinky'][1]
    sim_state.qpos[pinky_j2] = hand_pose['pinky'][2]
    sim_state.qpos[pinky_j3] = hand_pose['pinky'][3]
    # sim_state.qpos[pinky_j0] = 0.0
    # sim_state.qpos[pinky_j1] = 0.0
    # sim_state.qpos[pinky_j2] = 0.0
    # sim_state.qpos[pinky_j3] = 0.0

    sim.set_state(sim_state)

    # sim.forward()
    sim.step()
    assert sim.data.ncon == 0, 'Collision Detected!'
    # print("move hand to", hand_pose)
    # print_hand_joint_pose(sim)
    viewer.render()
    # assert sim.data.ncon == 0, 'Collision Detected!'
    time.sleep(0.001)



In [ ]:
# test for coordinate system transform(construct the thumb finger base plane normal vector)

# the mujoco world coordinate system(x:right/y:forward/z:up)
import numpy as np

vector1 = np.array([0,1,0])
# vector1 = np.array([0,0,1]) #the thumb plane normal vector at the condition that the thumb joint0 rotate axis is y
# method 1:(rotate target coordinate around reference coodinate by the sequence x, y and z axis)
# gamma = np.pi/4 #for x axis
# beta = -np.pi/4 #for y axis
# alpha = -np.pi/4 #for z axis

# # the sequence of rotation is x, y and z（at the mujoco coordinate system: x:right,y:backward,z:down）
# trans = np.matrix([[np.cos(alpha)*np.cos(beta), np.cos(alpha)*np.sin(beta)*np.sin(gamma)-np.sin(alpha)*np.cos(gamma), np.cos(alpha)*np.sin(beta)*np.cos(gamma)+np.sin(alpha)*np.sin(gamma)], [np.sin(alpha)*np.cos(beta), np.sin(alpha)*np.sin(beta)*np.sin(gamma)+np.cos(alpha)*np.cos(gamma), np.sin(alpha)*np.sin(beta)*np.cos(gamma)-np.cos(alpha)*np.sin(gamma)], [-np.sin(beta), np.cos(beta)*np.sin(gamma), np.cos(beta)*np.cos(gamma)]])
# print(trans)

# method 2:(euler rotate:rotate target coordinate around itself by the sequence z, y and x axis)
alpha = -np.pi/4 #for x axis
beta = 0.0 #for y axis
gamma = np.pi/4 #for z axis

Rx = np.matrix([[1, 0, 0], [0, np.cos(alpha), -np.sin(alpha)], [0, np.sin(alpha), np.cos(alpha)]])
Ry = np.matrix([[np.cos(beta), 0, np.sin(beta)], [0, 1, 0], [-np.sin(beta), 0, np.cos(beta)]])
Rz = np.matrix([[np.cos(gamma), -np.sin(gamma), 0], [np.sin(gamma), np.cos(gamma), 0],[0, 0, 1]])

# euler rotate(regular rotate sequence: zyx)
transform = Rx * Ry * Rz
print(transform)

vector2 = transform * vector1.reshape(3,1) #calc the new plane normal vector in the new coordinate（the joint rotate axis is still y）
print(vector2)



In [ ]:
# test for making dataset

# calc every hand pose joint value, mapping joint keypoints and record the image & data
%matplotlib inline

import sys
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/drive_hand")
import numpy as np
from data_importer.importer import Anyone
from hand_mapping import hand_keypoints_mapping
from joint_position_calculator import get_pose_new4

from numpy.testing import assert_array_equal,assert_almost_equal
from mujoco_py import MjSim, MjViewer, load_model_from_xml, load_model_from_path, MjSimState, ignore_mujoco_warnings
import mujoco_py
import cv2
from skimage.io import imsave, imshow
import matplotlib.pyplot as plt
import json
from funtest.test_pathlib import first_try
import time


# prepare the save file I/O
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/")
file_hand_ground_truth = open('./data/hand_groundtruth.txt','w+')
file_hand_mapping_keypoints = open('./data/hand_mapping_keyponts.txt','w+')
file_hand_calculated_joints = open('./data/hand_calculated_joints.txt','w+')

# prepare the mujcoco simulation environment
xml_path = "/home/jade/DRL/codes/MuJoCo/xml_model/MPL/robot_hand.xml"
model = load_model_from_path(xml_path)
sim = MjSim(model)
viewer = MjViewer(sim)

def print_hand_joint_pose(sim):
    print("hand pose:")
    print("wrist=", sim.data.get_joint_qpos("wrist_PRO"), sim.data.get_joint_qpos("wrist_UDEV"), sim.data.get_joint_qpos("wrist_FLEX"))
    print("thumb=", sim.data.get_joint_qpos("thumb_ABD"), sim.data.get_joint_qpos("thumb_MCP"), sim.data.get_joint_qpos("thumb_PIP"), sim.data.get_joint_qpos("thumb_DIP"))
    print("index=", sim.data.get_joint_qpos("index_ABD"), sim.data.get_joint_qpos("index_MCP"), sim.data.get_joint_qpos("index_PIP"), sim.data.get_joint_qpos("index_DIP"))
    print("middle=", sim.data.get_joint_qpos("middle_ABD"), sim.data.get_joint_qpos("middle_MCP"), sim.data.get_joint_qpos("middle_PIP"), sim.data.get_joint_qpos("middle_DIP"))
    print("ring=", sim.data.get_joint_qpos("ring_ABD"), sim.data.get_joint_qpos("ring_MCP"), sim.data.get_joint_qpos("ring_PIP"), sim.data.get_joint_qpos("ring_DIP"))
    print("pinky=", sim.data.get_joint_qpos("pinky_ABD"), sim.data.get_joint_qpos("pinky_MCP"), sim.data.get_joint_qpos("pinky_PIP"), sim.data.get_joint_qpos("pinky_DIP"))

print_hand_joint_pose(sim)

# get the ID of mechanical hand MPL in mujoco env
wrist_j0 = sim.model.get_joint_qpos_addr("wrist_PRO")
wrist_j1 = sim.model.get_joint_qpos_addr("wrist_UDEV")
wrist_j2 = sim.model.get_joint_qpos_addr("wrist_FLEX")

thumb_j0 = sim.model.get_joint_qpos_addr("thumb_ABD")
thumb_j1 = sim.model.get_joint_qpos_addr("thumb_MCP")
thumb_j2 = sim.model.get_joint_qpos_addr("thumb_PIP")
thumb_j3 = sim.model.get_joint_qpos_addr("thumb_DIP")

index_j0 = sim.model.get_joint_qpos_addr("index_ABD")
index_j1 = sim.model.get_joint_qpos_addr("index_MCP")
index_j2 = sim.model.get_joint_qpos_addr("index_PIP")
index_j3 = sim.model.get_joint_qpos_addr("index_DIP")

middle_j0 = sim.model.get_joint_qpos_addr("middle_ABD")
middle_j1 = sim.model.get_joint_qpos_addr("middle_MCP")
middle_j2 = sim.model.get_joint_qpos_addr("middle_PIP")
middle_j3 = sim.model.get_joint_qpos_addr("middle_DIP")

ring_j0 = sim.model.get_joint_qpos_addr("ring_ABD")
ring_j1 = sim.model.get_joint_qpos_addr("ring_MCP")
ring_j2 = sim.model.get_joint_qpos_addr("ring_PIP")
ring_j3 = sim.model.get_joint_qpos_addr("ring_DIP")

pinky_j0 = sim.model.get_joint_qpos_addr("pinky_ABD")
pinky_j1 = sim.model.get_joint_qpos_addr("pinky_MCP")
pinky_j2 = sim.model.get_joint_qpos_addr("pinky_PIP")
pinky_j3 = sim.model.get_joint_qpos_addr("pinky_DIP")

# prepare the image setting for the MPL hand in mujoco env
frame_size = [640, 480]
camera_name = "mpl_camera1"
device_id = 0

# make the iteration of the hand joints calculation through groundtruth dataset
for idx in range(1,20):
    try:
        # import groundtruth hand keypoints data
        ay = Anyone(idx)
        finger_data = ay.data.gtorig
        print("hand groundtruth keypoints:\n%s"%finger_data)

        # calc the mechanical hand keypoint mapping data and mechanical hand joints
        finger_data_mapping = hand_keypoints_mapping.calc_model_mpl_keypoints(finger_data)
        print("mechanical hand mapping keypoints:\n%s"%finger_data_mapping)
        hand_pose = get_pose_new4.get_hand_joint_pose(finger_data)
        print("calculated hand joints:\n%s"%hand_pose)

        # change and reset the state of mechanical hand MPL in mujoco env
        sim_state = sim.get_state()
        sim_state.qpos[wrist_j0] = np.pi
        sim_state.qpos[wrist_j1] = 0.0
        sim_state.qpos[wrist_j2] = 0.0

        sim_state.qpos[thumb_j0] = hand_pose['thumb'][0]
        sim_state.qpos[thumb_j1] = hand_pose['thumb'][1]
        sim_state.qpos[thumb_j2] = hand_pose['thumb'][2]
        sim_state.qpos[thumb_j3] = hand_pose['thumb'][3]

        sim_state.qpos[index_j0] = hand_pose['index'][0]
        sim_state.qpos[index_j1] = hand_pose['index'][1]
        sim_state.qpos[index_j2] = hand_pose['index'][2]
        sim_state.qpos[index_j3] = hand_pose['index'][3]

        sim_state.qpos[middle_j0] = hand_pose['middle'][0]
        sim_state.qpos[middle_j1] = hand_pose['middle'][1]
        sim_state.qpos[middle_j2] = hand_pose['middle'][2]
        sim_state.qpos[middle_j3] = hand_pose['middle'][3]

        sim_state.qpos[ring_j0] = hand_pose['ring'][0]
        sim_state.qpos[ring_j1] = hand_pose['ring'][1]
        sim_state.qpos[ring_j2] = hand_pose['ring'][2]
        sim_state.qpos[ring_j3] = hand_pose['ring'][3]

        sim_state.qpos[pinky_j0] = hand_pose['pinky'][0]
        sim_state.qpos[pinky_j1] = hand_pose['pinky'][1]
        sim_state.qpos[pinky_j2] = hand_pose['pinky'][2]
        sim_state.qpos[pinky_j3] = hand_pose['pinky'][3]

        sim.set_state(sim_state)

        # begin the simulation step and render
        sim.forward()
        sim.step()
        print("move hand to", hand_pose)
        print_hand_joint_pose(sim)
        
        # check whether the components of MPL happened collision
        assert sim.data.ncon == 0, 'MPL hand Collision Detected after stepping!'

        # render the mujoco scene and save the color/depth image of MPL into the file
        rgb_image, depth_image = sim.render(width = frame_size[0], height = frame_size[1], depth = True, mode = 'offscreen', camera_name = camera_name, device_id = device_id)

        # check whether the components of MPL happened collision
        assert sim.data.ncon == 0, 'MPL hand Collision Detected after rendering!'
        
        # format the finger data and write them into file
        str_finger_data = ','.join(map(str, finger_data))
        file_hand_ground_truth.write(str(idx) + ":" + str_finger_data + "\n")
        str_finger_data_mapping = ','.join(map(str, finger_data_mapping))
        file_hand_mapping_keypoints.write(str(idx) + ":" + str_finger_data_mapping + "\n")
        # str_hand_pose = map(str, hand_pose)
        str_hand_pose = str(hand_pose)
        file_hand_calculated_joints.write(str(idx) + ":" + str_hand_pose + "\n")

        # rotate the image along the u axis(horizontally flip) and save the rgb image
        rgb_image = cv2.flip(rgb_image, 0)
        print(np.shape(rgb_image))
        cv2.imwrite("./data/mpl_rgb_%d.jpg"%idx, rgb_image)
        # cv2.imshow(rgb_image)
        plt.imshow(rgb_image)
        plt.show()

        # rotate the image along the u axis(horizontally flip) and save the depth image
        depth_image = cv2.flip(depth_image, 0)
        print(np.shape(depth_image))
        # translate the value range into range[0,255]
        dmin = np.amin(depth_image)
        dmax = np.amax(depth_image)
        scale = (dmax-dmin) / 255.0
        depth_image = (depth_image - dmin) / scale
        cv2.imwrite("./data/mpl_depth_%d.png"%idx, depth_image)
        # cv2.imshow(depth_image)
        plt.imshow(depth_image)
        plt.show()

        # time.sleep(0.002)

    except:
        # raise
        print("Components of MPL Hand in MuJoCo scene have collided, so we exclude the data sample and keep going")
        # pass
        continue
    
    else:
        print("No collision happened in MuJoCo scene, so let's run the next data sample")

# finally 
file_hand_ground_truth.close()
file_hand_mapping_keypoints.close()
file_hand_calculated_joints.close()
print("haha, all dataset sample have been checked, well done!")

    